In [2]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [3]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    print("error")

In [4]:
# Create the variables x and y and their bounds.
x = solver.NumVar(0, 1, 'x')   # bound 0 <= x <= 1
y = solver.NumVar(0, 2, 'y')   # bound 0 <= y <= 2

print('Number of variables =', solver.NumVariables())

Number of variables = 2


In [5]:
# Create a linear constraint, 0 <= x + y <= 2.
ct = solver.Constraint(0, 2, 'ct')
ct.SetCoefficient(x, 1)
ct.SetCoefficient(y, 1)

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 1


In [6]:
# Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

In [7]:
solver.Solve()
print('Solution:')
print('Objective value =', objective.Value())
print('x =', x.solution_value())
print('y =', y.solution_value())

Solution:
Objective value = 4.0
x = 1.0
y = 1.0


# facility location problem 

See https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/facility_location/facility_location_gcl.ipynb

In [8]:
from math import sqrt
from itertools import product

In [9]:
# Parameters
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

In [10]:
# This function determines the Euclidean distance between a facility and customer sites.

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

In [11]:
# Compute key parameters of MIP model formulation

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

In [12]:
# Compute shipping costs

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

In [13]:
shipping_cost

{(0, 0): 1.5,
 (0, 1): 0.5,
 (0, 2): 0.5,
 (0, 3): 1.8027756377319946,
 (0, 4): 1.118033988749895,
 (0, 5): 1.118033988749895,
 (0, 6): 2.5,
 (0, 7): 2.0615528128088303,
 (0, 8): 2.0615528128088303,
 (1, 0): 2.773084924772409,
 (1, 1): 2.5079872407968904,
 (1, 2): 2.6248809496813377,
 (1, 3): 1.9209372712298547,
 (1, 4): 1.5132745950421556,
 (1, 5): 1.7,
 (1, 6): 1.3,
 (1, 7): 0.5385164807134504,
 (1, 8): 0.9433981132056605}

In [38]:
# solver = pywraplp.Solver.CreateSolver('CBC_MIXED_INTEGER_PROGRAMMING')
solver = pywraplp.Solver.CreateSolver('SCIP_MIXED_INTEGER_PROGRAMMING')

In [39]:
select = {}
for j in range(num_facilities):
    select[j] = solver.BoolVar("Select")

In [40]:
assign = {}
for i,j in shipping_cost.keys(): 
    assign[(i,j)] = solver.NumVar(0,1,"Assign")

In [41]:
# shipping constraints
for i,j in cartesian_prod:
    solver.Add(assign[(i,j)] <= select[j])

In [42]:
# demand constraints
for i in range(num_customers):
    con = solver.Constraint(1,1)
    for j in range(num_facilities):
        con.SetCoefficient(assign[(i,j)], 1)  

In [43]:
objective = solver.Objective()

In [44]:
for j in range(num_facilities):
    objective.SetCoefficient(select[j], setup_cost[j])

In [45]:
for i in range(num_customers):
    for j in range(num_facilities):
        objective.SetCoefficient(assign[(i,j)],shipping_cost[(i,j)])

In [46]:
objective.SetMinimization()

In [55]:
status = solver.Solve()

In [56]:
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 4.723712908961849

Problem solved in 620000.000000 milliseconds
Problem solved in 19 iterations
Problem solved in 1 branch-and-bound nodes


In [63]:
# display optimal values of decision variables

for facility in select.keys():
    if (abs(select[facility].solution_value()) > 1e-6):
        print(f"\n Build a warehouse at location {facility + 1}.")


 Build a warehouse at location 4.


In [60]:
# Shipments from facilities to customers.

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].solution_value()) > 1e-6):
        print(f"\n Supermarket {customer + 1} receives {round(100*assign[customer, facility].solution_value(), 2)} % of its demand  from Warehouse {facility + 1} .")


 Supermarket 1 receives 100.0 % of its demand  from Warehouse 4 .

 Supermarket 2 receives 100.0 % of its demand  from Warehouse 4 .
